# Prerequisites

In [ ]:
# TensorFlow Extended와 TensorFlow가 깔려 있다면 이 셀은 스킵.
!pip install tfx
!pip install tensorflow

In [1]:
import os
import pathlib
import shutil

import tensorflow as tf
import tensorflow_model_analysis as tfma
import tfx
from tfx.components import (ImportExampleGen,
                            StatisticsGen, SchemaGen, ExampleValidator,
                            Transform, Trainer, Evaluator, Pusher)
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.v1.dsl import Resolver
from tfx.v1.dsl.experimental import LatestBlessedModelStrategy

2023-01-14 05:15:59.999602: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
def copy_artifact(component, root_dir):
    for key, output in component.outputs.items():
        src = output.get()[0].uri
        dest = pathlib.Path(os.path.join(root_dir, key))
        shutil.copytree(src, dest, dirs_exist_ok=True)

In [16]:
record_dir = pathlib.Path("data/complaints/records")
artifact_dir = pathlib.Path("artifacts")
transform_module = "transform_module.py"
trainer_module = "trainer_module.py"
_serving_model_dir = "serving_model_dir"

# Pipeline

In [4]:
context = InteractiveContext()

## 1. ExampleGen

In [10]:
example_gen = ImportExampleGen(input_base=str(record_dir))
context.run(example_gen)
examples = example_gen.outputs["examples"]
copy_artifact(example_gen, artifact_dir)

## 2. StatisticsGen

In [11]:
statistics_gen = StatisticsGen(examples=examples)
context.run(statistics_gen)
stats = statistics_gen.outputs["statistics"]
copy_artifact(statistics_gen, artifact_dir)

## 3. SchemaGen

In [12]:
schema_gen = SchemaGen(statistics=stats, infer_feature_shape=True)
context.run(schema_gen)
schema = schema_gen.outputs["schema"]
copy_artifact(schema_gen, artifact_dir)

## 4. ExampleValidator

In [13]:
example_validator = ExampleValidator(statistics=stats, schema=schema)
context.run(example_validator)
anomalies = example_validator.outputs["anomalies"]
copy_artifact(example_validator, artifact_dir)

## 5. Transform

In [15]:
transform = Transform(examples=examples,
                      schema=schema,
                      module_file=transform_module)
context.run(transform)
transform_graph = transform.outputs["transform_graph"]
transformed_examples = transform.outputs["transformed_examples"]
copy_artifact(transform, artifact_dir)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying transform_module.py -> build/lib
copying trainer_module.py -> build/lib
installing to /tmp/tmpz0dauhlz
running install
running install_lib
copying build/lib/transform_module.py -> /tmp/tmpz0dauhlz
copying build/lib/trainer_module.py -> /tmp/tmpz0dauhlz
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpz0dauhlz/tfx_user_code_Transform-0.0+8d583d042595efa91bdc0a51985ccb9118c9b9a0a3ae81b0eb56cfc5d1c3be64-py3

/home/hakjun/projects/pipeline/venv/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


Processing /tmp/tfx-interactive-2023-01-14T05_16_20.442478-w492wnpr/_wheels/tfx_user_code_Transform-0.0+8d583d042595efa91bdc0a51985ccb9118c9b9a0a3ae81b0eb56cfc5d1c3be64-py3-none-any.whl
Processing /tmp/tfx-interactive-2023-01-14T05_16_20.442478-w492wnpr/_wheels/tfx_user_code_Transform-0.0+8d583d042595efa91bdc0a51985ccb9118c9b9a0a3ae81b0eb56cfc5d1c3be64-py3-none-any.whl
Processing /tmp/tfx-interactive-2023-01-14T05_16_20.442478-w492wnpr/_wheels/tfx_user_code_Transform-0.0+8d583d042595efa91bdc0a51985ccb9118c9b9a0a3ae81b0eb56cfc5d1c3be64-py3-none-any.whl


2023-01-14 05:19:03.861400: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
Use ref() instead.


...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


## 6. Trainer

In [ ]:
trainer = Trainer(examples=transformed_examples,
                  transform_graph=transform_graph,
                  schema=schema, module_file=trainer_module,
                  train_args=tfx.proto.trainer_pb2.TrainArgs(num_steps=50),
                  eval_args=tfx.proto.trainer_pb2.EvalArgs(num_steps=10))
context.run(trainer)
trained_model = trainer.outputs["model"]
copy_artifact(trainer, artifact_dir)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying transform_module.py -> build/lib
copying trainer_module.py -> build/lib
installing to /tmp/tmpy7izb9_f
running install
running install_lib
copying build/lib/transform_module.py -> /tmp/tmpy7izb9_f
copying build/lib/trainer_module.py -> /tmp/tmpy7izb9_f
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpy7izb9_f/tfx_user_code_Trainer-0.0+8d583d042595efa91bdc0a51985ccb9118c9b9a0a3ae81b0eb56cfc5d1c3be64-py3.8.egg-info
runnin

/home/hakjun/projects/pipeline/venv/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


Processing /tmp/tfx-interactive-2023-01-14T05_16_20.442478-w492wnpr/_wheels/tfx_user_code_Trainer-0.0+8d583d042595efa91bdc0a51985ccb9118c9b9a0a3ae81b0eb56cfc5d1c3be64-py3-none-any.whl


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


50/50 [==============================] - ETA: 0s - loss: 0.5889 - binary_accuracy: 0.7225 - true_positives: 17.0000

## 7. Resolver

In [ ]:
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
).with_id("latest_blessed_model_resolver")
context.run(model_resolver)
copy_artifact(model_resolver, artifact_dir)

## 8. Evaluator

In [ ]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key="consumer_disputed")],
    slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=["product"])],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name="BinaryAccuracy"),
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(class_name="AUC")],
            # baseline 모델과 비교해 우위에 있더라도 아래 임계치를 넘어야 bless를 받는다.
            thresholds={
                "AUC": tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={"value": 0.65}),
                    # 두 모델 간 지표 ∆가 0.01은 넘어야 하고, 새 모델 지표값은 클수록 좋다는 의미.
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={"value": 0.01}))})])

In [ ]:
evaluator = Evaluator(
    examples=examples,
    model=trained_model,
    baseline_model=model_resolver.outputs["model"],
    eval_config=eval_config)
context.run(evaluator)
blessing = evaluator.outputs["blessing"]
evaluation = evaluator.outputs["evaluation"]
copy_artifact(evaluator, artifact_dir)

In [ ]:
output_path = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(output_path)
validation_result = tfma.load_validation_result(output_path)

## 9. Pusher

In [ ]:
pusher = Pusher(model=trained_model,
                model_blessing=blessing,
                push_destination=pusher_pb2.PushDestination(
                    filesystem=pusher_pb2.PushDestination.Filesystem(base_directory=_serving_model_dir)))
context.run(pusher)
pushed_model = pusher.outputs["pushed_model"]
copy_artifact(pusher, artifact_dir)